In [1]:
import pandas as pd 
import re
import numpy as np
from tqdm import tqdm

valid_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/valid_json_audio_data_decoder_time_arrest_stack.csv')
test_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/test_json_audio_data_decoder_time_arrest_stack.csv')

sentence_valid = valid_df.text.tolist()
sentence_test = test_df.text.tolist()

/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_52683/4039726886.py:6: DtypeWarning: Columns (170,174,178,182,186,190,194,198,202,206,210,214,218,222,226,230,234,238,242,246,250,254,258,262,266,270,274,278,282,286,290,294,298,302,306,310,314,318,322,326,330,334) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/valid_json_audio_data_decoder_time_arrest_stack.csv')
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_52683/4039726886.py:7: DtypeWarning: Columns (190,194,198,202,206,210,214,218,222,226,230,234,238,242,246,250,254,258,262,266,270,274,278,282,286,290,294,298,302,306,310,314,318,322,326,330,334,338,342,346) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/test_json_audio_data_decoder_time_arrest_stack.csv')


In [2]:
df = pd.read_csv('/Users/dongjoon/Documents/bow/not_stacked/frequnecy_df_cum.csv',index_col=0)
word_list = df.Word.tolist()

In [3]:
def generate_bows_sen(sentence , word_list):

    bag_vectors = np.zeros(len(word_list))
    for i, word in enumerate(word_list):
        word_list=[]
        word_count =  len(re.findall(str(word), str(sentence)))
        bag_vectors[i] += word_count
    return bag_vectors.tolist()     # 한 EHR_DX row 에 들어있는 token 개수, bow count한 row -> dict_list 개수 +1 

In [4]:
def main(EHR, words_list, type):
    bows = []
    for row in tqdm(EHR):
        bow = generate_bows_sen(row, words_list)
        bows.append(bow)
    bow_df = pd.DataFrame(bows)
    return bow_df


In [5]:
valid_bow=main(sentence_valid, word_list, 'valid')
test_bow=main(sentence_test, word_list, 'test')

100%|██████████| 902847/902847 [1:50:49<00:00, 135.78it/s]  


In [6]:
valid_bow.columns = word_list 
print('export')
valid_bow.to_csv('./bow_df_valid_new.csv', encoding='utf-8-sig')

test_bow.columns = word_list
print('export')
test_bow.to_csv('./bow_df_test_new.csv', encoding='utf-8-sig')

export
export


In [1]:
import pandas as pd

In [2]:
valid_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/valid_json_audio_data_decoder_time_arrest_stack.csv')
test_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/test_json_audio_data_decoder_time_arrest_stack.csv')

valid_sentence = valid_df.text.tolist()
test_sentence = test_df.text.tolist()

valid_id = valid_df[['id']].reset_index(drop=True)
valid_sid = valid_df[['s_id']].reset_index(drop=True)
test_id = test_df[['id']].reset_index(drop=True)
test_sid = test_df[['s_id']].reset_index(drop=True)

/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_74751/658820105.py:1: DtypeWarning: Columns (170,174,178,182,186,190,194,198,202,206,210,214,218,222,226,230,234,238,242,246,250,254,258,262,266,270,274,278,282,286,290,294,298,302,306,310,314,318,322,326,330,334) have mixed types. Specify dtype option on import or set low_memory=False.
  valid_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/valid_json_audio_data_decoder_time_arrest_stack.csv')
/var/folders/r2/dldm7t_d37nfdthx_29x_thc0000gn/T/ipykernel_74751/658820105.py:2: DtypeWarning: Columns (190,194,198,202,206,210,214,218,222,226,230,234,238,242,246,250,254,258,262,266,270,274,278,282,286,290,294,298,302,306,310,314,318,322,326,330,334,338,342,346) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('/Users/dongjoon/Documents/bow/stacked/test_json_audio_data_decoder_time_arrest_stack.csv')


In [3]:
valid_tf = pd.read_csv('./bow_df_valid_new.csv')
valid_tf.drop(['Unnamed: 0'], axis = 1, inplace = True)

test_tf = pd.read_csv('./bow_df_test_new.csv')
test_tf.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [4]:
df_1 = valid_tf.astype(bool).sum(axis = 0)
df_2 = test_tf.astype(bool).sum(axis=0)

In [5]:
valid_idf = pd.read_csv('/Users/dongjoon/Documents/bow/not_stacked/train_idf_df.csv',index_col=0)['idf']
test_idf = pd.read_csv('/Users/dongjoon/Documents/bow/not_stacked/train_idf_df.csv',index_col=0)['idf']

In [6]:
import numpy as np

# 문서 개수
valid_D = len(valid_tf)

# Inverse Document Frequency
# valid_idf = np.log((valid_D+1) / (df_1+1)) + 1

valid_tfidf = valid_tf * valid_idf                      
valid_norm = np.linalg.norm(valid_tfidf, axis=1, keepdims=True)
valid_norm[valid_norm == 0] = 1e-10  # Replace zero norms with a small number
valid_tfidf = valid_tfidf / valid_norm

valid_tfidf = pd.concat([valid_tfidf, valid_id], axis=1)
valid_tfidf

,전화,선생,주소,구급차,병원,길,의식,앞,아파트,말씀,...,인기척,소아과,진행,자해,방화,부인,음식,극동,광명,id
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aa761
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aa761
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aa761
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aa761
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aa761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725187,0.0,0.0,0.126880,0.137820,0.0,0.0,0.0,0.174165,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651e4a3786dc055ca8bf4604
725188,0.0,0.0,0.097132,0.105507,0.0,0.0,0.0,0.133331,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651e4a3786dc055ca8bf4604
725189,0.0,0.0,0.097132,0.105507,0.0,0.0,0.0,0.133331,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651e4a3786dc055ca8bf4604
725190,0.0,0.0,0.096365,0.104674,0.0,0.0,0.0,0.132278,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,651e4a3786dc055ca8bf4604


In [7]:
import numpy as np

# 문서 개수
test_D = len(test_tf)

# Inverse Document Frequency
# test_idf = np.log((test_D+1) / (df_2+1)) + 1

test_tfidf = test_tf * test_idf                      
test_norm = np.linalg.norm(test_tfidf, axis=1, keepdims=True)
test_norm[test_norm == 0] = 1e-10  # Replace zero norms with a small number
test_tfidf = test_tfidf / test_norm

test_tfidf = pd.concat([test_tfidf, test_id], axis=1)
test_tfidf

,전화,선생,주소,구급차,병원,길,의식,앞,아파트,말씀,...,인기척,소아과,진행,자해,방화,부인,음식,극동,광명,id
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64f6c516461a963f30a3c498
1,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64f6c516461a963f30a3c498
2,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64f6c516461a963f30a3c498
3,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64f6c516461a963f30a3c498
4,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64f6c516461a963f30a3c498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
902842,0.0,0.144748,0.0,0.0,0.039636,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aba03
902843,0.0,0.144748,0.0,0.0,0.039636,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aba03
902844,0.0,0.144190,0.0,0.0,0.039483,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aba03
902845,0.0,0.144190,0.0,0.0,0.039483,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64dd752b1ef84058319aba03


In [8]:
valid_tfidf.to_csv('valid_tf_idf(train_idf).csv',encoding='utf-8-sig')
test_tfidf.to_csv('test_tf_idf(train_idf).csv',encoding='utf-8-sig')